# Kaggle Titanic 3eme essai !

## Jeu de test

In [26]:
import pandas as pd
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler

train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

### Travail sur le ticket

In [27]:
noticket = []
test['Ticket'].fillna('X')
for ticketnn in test['Ticket']:
    if (ticketnn == 'X'):
        noticket.append(1)
    else:
        noticket.append(0)
pd.DataFrame(noticket)[0].value_counts()

0    418
Name: 0, dtype: int64

####  Tout le monde a un ticket !

In [28]:
test['Ticket'].value_counts().head()

PC 17608              5
113503                4
CA. 2343              4
SOTON/O.Q. 3101315    3
16966                 3
Name: Ticket, dtype: int64

####  Par contre les tickets ne sont pas uniques ! il va donc falloir diviser le prix par nombre de ticket

In [29]:
# Inutile / juste pour un exemple ;-)
#listunit = pd.DataFrame(test.groupby(['Ticket'])['Fare'].mean())
#listunit['Ticket'] = listunit.index

# Prépartion d'un DF (TicketCounts) contenant les ticket avec leur nb d'occurence
TicketCounts = pd.DataFrame(test['Ticket'].value_counts().head())
TicketCounts['TicketCount'] = TicketCounts['Ticket'] # renomme la colonne Ticket
TicketCounts['Ticket'] = TicketCounts.index # rajoute une colonne Ticket pour le merge (jointure)

# Reporte le résultat dans le dataframe test (jointure des datasets)
fin = pd.merge(test, TicketCounts, how='left', on='Ticket')
fin['PrixUnitaire'] = fin['Fare'] / fin['TicketCount'].fillna(1)

#prxunit = pd.DataFrame(fin['PrixUnitaire'])
fin

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,TicketCount,PrixUnitaire
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,NaN,7.829200
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,NaN,7.000000
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,NaN,9.687500
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,NaN,8.662500
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,NaN,12.287500
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S,NaN,9.225000
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q,NaN,7.629200
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S,NaN,29.000000
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C,NaN,7.229200
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S,NaN,24.150000


#### Attention on a un passager qui n'a pas de prix de ticket

In [30]:
import numpy as np
test.loc[np.isnan(test['Fare'])]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S


In [31]:
# prennons la moyenne des embarqués à S... !
test.loc[test['Embarked'] == 'S']['Fare'].mean()

28.230436431226774

### Travail sur le nom

In [32]:
familynames = []
for noms in test["Name"]:
    familynames.append(re.search('([A-Za-z0-9]*),\ ([A-Za-z0-9 ]*)\. (.*)', noms).group(1))
pdfamilynames = pd.DataFrame(familynames, columns = ['familynames'])

famsurv = test.join(pdfamilynames)
#famsurv['familynames'].value_counts()

### Travail sur le titre

In [33]:
X = train
X['Titre'] = X.Name.map(lambda x : x.split(",")[1].split(".")[0])
X['NomFamille'] = X.Name.map(lambda x : x.split(",")[0])
titre = pd.DataFrame(X['Titre'])
X['Titre'].value_counts()

 Mr              517
 Miss            182
 Mrs             125
 Master           40
 Dr                7
 Rev               6
 Major             2
 Col               2
 Mlle              2
 Jonkheer          1
 Mme               1
 Ms                1
 the Countess      1
 Lady              1
 Sir               1
 Capt              1
 Don               1
Name: Titre, dtype: int64

In [34]:
X['iTitre'] = 0
for idx, titre in enumerate(X['Titre']): 
    if (titre.strip() == 'Mr') :
        X.loc[idx, 'iTitre'] = 2
    elif (titre.strip() == 'Miss' or titre.strip() == 'Mrs') :
        X.loc[idx, 'iTitre'] = 1
    else :
        X.loc[idx, 'iTitre'] = 0
X['iTitre'].value_counts()

2    517
1    307
0     67
Name: iTitre, dtype: int64

## Création d'une fonction de préparation

In [35]:
def dataprep(data):
    # Sexe
    sexe = pd.get_dummies(data['Sex'], prefix='sex')
    
    # Cabine, récupération du pont
    cabin = pd.get_dummies(data['Cabin'].fillna('X').str[0], prefix='Cabin')
    
    # Age 
    age = data['Age'].fillna(data['Age'].mean())
    enfantf = []
    for i in range(data.shape[0]) :
        if age[i] < 9:
            enfantf.append(0)
        else:
            enfantf.append(1)
    enfantd = pd.DataFrame(enfantf, columns = ['Enfant'])
    
    # Titre
    data['Titre'] = data.Name.map(lambda x : x.split(",")[1].split(".")[0]).fillna('X')
    data['iTitre'] = 0
    for idx, titre in enumerate(data['Titre']): 
        if (titre.strip() == 'Mr') :
            data.loc[idx, 'iTitre'] = 2
        elif (titre.strip() == 'Miss' or titre.strip() == 'Mrs') :
            data.loc[idx, 'iTitre'] = 1
        else :
            data.loc[idx, 'iTitre'] = 0

    # Embarquement
    emb = pd.get_dummies(data['Embarked'], prefix='emb')
    
    # Prix unitaire - Ticket, Prépartion d'un DF (TicketCounts) contenant les ticket avec leur nb d'occurence
    TicketCounts = pd.DataFrame(data['Ticket'].value_counts().head())
    TicketCounts['TicketCount'] = TicketCounts['Ticket'] # renomme la colonne Ticket
    TicketCounts['Ticket'] = TicketCounts.index # rajoute une colonne Ticket pour le merge (jointure)
    # reporte le résultat dans le dataframe test (jointure des datasets)
    fin = pd.merge(data, TicketCounts, how='left', on='Ticket')
    fin['PrixUnitaire'] = fin['Fare'] / fin['TicketCount'].fillna(1)
    prxunit = pd.DataFrame(fin['PrixUnitaire'])
    
    # Classe
    pc = pd.DataFrame(MinMaxScaler().fit_transform(data[['Pclass']]), columns = ['Classe'])

    dp = data[['SibSp', 'Parch', 'iTitre']].join(pc).join(sexe).join(emb).join(prxunit).join(cabin).join(age).join(enfantd)
    return dp

In [36]:
Xtrain = dataprep(train)
# remove the Cabin with T value as this does not exist in the test dataset !
del Xtrain['Cabin_T']
Xtest = dataprep(test)
Xtest = Xtest.fillna(28)

In [37]:
Xtrain.describe(include='all')

,SibSp,Parch,iTitre,Classe,sex_female,sex_male,emb_C,emb_Q,emb_S,PrixUnitaire,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_X,Age,Enfant
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.523008,0.381594,1.505051,0.654321,0.352413,0.647587,0.188552,0.086420,0.722783,30.765530,0.016835,0.052750,0.066218,0.037037,0.035915,0.014590,0.004489,0.771044,29.699118,0.939394
std,1.102743,0.806057,0.633101,0.418036,0.477990,0.477990,0.391372,0.281141,0.447876,49.752483,0.128725,0.223659,0.248802,0.188959,0.186182,0.119973,0.066890,0.420397,13.002015,0.238740
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.420000,0.000000
25%,0.000000,0.000000,1.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,7.895800,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,22.000000,1.000000
50%,0.000000,0.000000,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,29.699118,1.000000
75%,1.000000,0.000000,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,28.856250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,35.000000,1.000000
max,8.000000,6.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,512.329200,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,80.000000,1.000000


## Entrainement du modèle

In [39]:
y = train.Survived

#clf = LinearSVC(random_state=2)
#clf.fit(Xtrain, y)
#p_tr = clf.predict(Xtrain)
#print ("Score Train -->", round(clf.score(Xtrain, y) *100,4), " %")

rf = RandomForestClassifier(n_estimators=100, random_state=0, max_features=2)
rf.fit(Xtrain, y)
p_tr = rf.predict(Xtrain)
print ("Score Train -->", round(rf.score(Xtrain, y) *100,4), " %")

Score Train --> 98.7654  %


## Formattage des résultats pour Kaggle ;-)

In [42]:
Xtest.describe()

,SibSp,Parch,iTitre,Classe,sex_female,sex_male,emb_C,emb_Q,emb_S,PrixUnitaire,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_X,Age,Enfant
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,0.447368,0.392344,1.507177,0.632775,0.363636,0.636364,0.244019,0.110048,0.645933,30.371621,0.016746,0.043062,0.083732,0.031100,0.021531,0.019139,0.002392,0.782297,30.272590,0.956938
std,0.896760,0.981429,0.620355,0.420919,0.481622,0.481622,0.430019,0.313324,0.478803,45.930529,0.128474,0.203240,0.277317,0.173797,0.145320,0.137177,0.048912,0.413179,12.634534,0.203240
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.170000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.881275,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,23.000000,1.000000
50%,0.000000,0.000000,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,14.454200,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,30.272590,1.000000
75%,1.000000,0.000000,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,30.375000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,35.750000,1.000000
max,8.000000,9.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,512.329200,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,76.000000,1.000000


In [43]:
p_test = clf.predict(Xtest)
result = pd.DataFrame(test['PassengerId'])
pred = pd.DataFrame(p_test, columns=['Survived'])
result = result.join(pred)

In [46]:
result.describe()

,PassengerId,Survived
count,418.000000,418.000000
mean,1100.500000,0.277512
std,120.810458,0.448308
min,892.000000,0.000000
25%,996.250000,0.000000
50%,1100.500000,0.000000
75%,1204.750000,1.000000
max,1309.000000,1.000000


In [48]:
result.to_csv("./data/result3.csv", columns=["PassengerId", "Survived"], index=False)

# Résultat : score = 